In [3]:
import neo4j
import osmnx as ox
import geopy.distance

In [2]:
G = ox.load_graphml("./data/krakow.graphml")

In [3]:
print(G)

MultiDiGraph with 333120 nodes and 721628 edges


In [22]:
gdf_nodes, gdf_relationships = ox.graph_to_gdfs(G)
gdf_nodes.reset_index(inplace=True)
gdf_relationships.reset_index(inplace=True)

In [23]:
gdf_nodes["x"] = gdf_nodes.apply(lambda x: x.geometry.coords.xy[0][0], axis=1)
gdf_nodes["y"] = gdf_nodes.apply(lambda x: x.geometry.coords.xy[1][0], axis=1)
gdf_nodes = gdf_nodes.drop(columns=["street_count", "geometry", "highway", "ref"])

In [24]:
gdf_nodes

,osmid,y,x
0,13822575,50.082051,20.032569
1,13822576,50.080695,20.034350
2,13822577,50.079400,20.036068
3,13822581,50.081723,20.036520
4,13822694,50.081361,20.035596
...,...,...,...
333115,11286442324,50.063331,19.960819
333116,11286442325,50.063072,19.961152
333117,11286442326,50.063120,19.960761
333118,11286442327,50.063135,19.961028


In [30]:
gdf_nodes.to_csv("./data/krakow_nodes_neo4j.csv", index=False)

In [26]:
gdf_relationships = gdf_relationships.drop(columns=["oneway", "ref", "highway", "maxspeed", "reversed", "junction", "service", "key", 
                                                    "access", "width", "bridge", "est_width", "tunnel", "geometry", "lanes", "name"])

In [27]:
gdf_relationships

,u,v,osmid,length
0,13822575,965423962,25042019,18.886
1,13822575,8284483157,148469683,8.987
2,13822575,777395552,997754503,13.867
3,13822576,1234198750,2954554,14.652
4,13822576,5339146823,553072298,24.027
...,...,...,...,...
721623,11286442326,5773366079,1218055223,14.989
721624,11286442326,1919699663,1218055223,3.619
721625,11286442327,11286442303,609536558,7.807
721626,11286442327,11286442326,609536558,19.125


In [31]:
gdf_relationships.to_csv("./data/krakow_relationships_neo4j.csv", index=False)

### Search for good osmid pairs

In [9]:
# Node 1 - zakamycze
node_nr_1 = 356926768
gdf_nodes[gdf_nodes["osmid"] == node_nr_1]

,osmid,y,x,street_count,highway,ref,geometry
19027,356926768,50.060711,19.838658,2,NaN,NaN,POINT (19.83866 50.06071)


In [10]:
# Node 2 - betel
node_nr_2 = 2104495834
gdf_nodes[gdf_nodes["osmid"] == node_nr_2]

,osmid,y,x,street_count,highway,ref,geometry
78191,2104495834,50.063631,19.934842,3,NaN,NaN,POINT (19.93484 50.06363)


In [11]:
# two random osmid
start_and_finish = gdf_nodes.sample(2)

osmid_1 = start_and_finish.iloc[0]["osmid"]
osmid_2 = start_and_finish.iloc[1]["osmid"]

coords_1 = start_and_finish.iloc[0]["y"], start_and_finish.iloc[0]["x"]
coords_2 = start_and_finish.iloc[1]["y"], start_and_finish.iloc[1]["x"]

print(f"osmid_1: {osmid_1}, osmid_2: {osmid_2}, distance: {geopy.distance.distance(coords_1, coords_2).km} km")

osmid_1: 1924205909, osmid_2: 11098530035, distance: 5.788148236128053 km


## Neo4j Import

In [21]:
NEO4J_URI = "bolt://localhost:7690"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "graph-routing"
NEO4J_DATABASE = "neo4j"

driver = neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD), database=NEO4J_DATABASE)

In [22]:
clear_data_query = "MATCH (n) DETACH DELETE n"

clear_indexes_and_constrains = "CALL apoc.schema.assert({}, {}, true) YIELD label, key RETURN *"

node_constraint_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (i:Intersection) REQUIRE i.osmid IS UNIQUE"

rel_index_query = "CREATE INDEX IF NOT EXISTS FOR ()-[r:ROAD_SEGMENT]-() ON r.osmid"
    
nodes_csv_load = """
    LOAD CSV WITH HEADERS FROM 
    "file:///krakow_nodes_neo4j.csv" AS row
    WITH row WHERE row.osmid IS NOT NULL
    MERGE (i:Intersection {osmid: toInteger(row.osmid)})
    SET i.latitude = toFloat(row.y), 
        i.longitude = toFloat(row.x)
    RETURN COUNT(*) as total
    """
    
relationships_csv_load = """
    LOAD CSV WITH HEADERS FROM 
    "file:///krakow_relationships_neo4j.csv" AS row
    MATCH (from:Intersection {osmid: toInteger(row.u)})
    MATCH (to:Intersection {osmid: toInteger(row.v)})
    MERGE (from)-[r:ROAD_SEGMENT {osmid: toInteger(row.osmid)}]->(to)
    SET r.length = toFloat(row.length)
    RETURN COUNT(*) AS total
    """


In [31]:
def print_result(results):
    result = [dict(i) for i in results]
    print(result)
    
    result_all = results.consume()
    print(f"result_consumed_after: {result_all.result_consumed_after}")
    print(f"result_available_after: {result_all.result_available_after}\n")

In [24]:
def delete_database(tx):
    results = tx.run(clear_data_query)
    print_result(results)

def clear_indexes(tx):
    results = tx.run(clear_indexes_and_constrains)
    print_result(results)

In [25]:
def create_constraints(tx):
    results = tx.run(node_constraint_query)
    print_result(results)

    results = tx.run(rel_index_query)
    print_result(results)

In [26]:
def load_nodes(tx):
    results = tx.run(nodes_csv_load)
    print_result(results)

In [27]:
def load_relationshpis(tx):
    results = tx.run(relationships_csv_load)
    print_result(results)

### Clear database

In [32]:
with driver.session() as session:
    session.execute_write(delete_database)
    
with driver.session() as session:
    session.execute_write(clear_indexes)

[]
result_consumed_after: 0
result_available_after: 76

[]
result_consumed_after: 2
result_available_after: 3



### Create constraints

In [94]:
with driver.session() as session:
    session.execute_write(create_constraints)

### Load data

In [95]:
with driver.session() as session:
    session.execute_write(load_nodes)

In [96]:
with driver.session() as session:
    session.execute_write(load_relationshpis)

### Simple search path query

```
EXPLAIN PROFILE
```

```
MATCH (source:Intersection {osmid: 356926768}) 
MATCH (target:Intersection {osmid: 2104495834})
CALL apoc.algo.dijkstra(source, target, "ROAD_SEGMENT", "length")
YIELD path, weight
RETURN path, weight
```

```
MATCH (source:Intersection {osmid: 356926768}) 
MATCH (target:Intersection {osmid: 2104495834})
CALL apoc.algo.aStar(source, target, "ROAD_SEGMENT", "length", "latitude", "longitude")
YIELD path, weight
RETURN path, weight
```

In [5]:
def dijkstra_query(tx, osmid_1, osmid_2):
    dijkstra_query = f"""
        MATCH (source:Intersection {{osmid: {osmid_1}}}) 
        MATCH (target:Intersection {{osmid: {osmid_2}}})
        CALL apoc.algo.dijkstra(source, target, "ROAD_SEGMENT", "length")
        YIELD path, weight
        RETURN path, weight
        """
    results = tx.run(dijkstra_query)
    print_result(results)

In [6]:
node_nr_1 = 356926768
node_nr_2 = 2104495834

In [7]:
with driver.session() as session:
    session.execute_write(dijkstra_query, node_nr_1, node_nr_2)

AttributeError: 'Result' object has no attribute 'summary'

### Add addresses - TODO

In [ ]:
# We'll use apoc.load.json to import a JSON file of address data

address_constraint_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (a:Address) REQUIRE a.id IS UNIQUE"

add_addresses_query = """
CALL apoc.periodic.iterate(
  'CALL apoc.load.json("/home/pcend/Piotr/kod/lisboa-routing/data/lisboa.geojson") YIELD value',
  'MERGE (a:Address {id: value.properties.id})
SET a.location = 
  point(
      {latitude: value.geometry.coordinates[1], longitude: value.geometry.coordinates[0]}),
    a.full_address = value.properties.number + " " + value.properties.street + " " + value.properties.city + ", CA " + value.properties.postcode

SET a += value.properties',
  {batchSize:10000, parallel:true})
"""

# Next, connect each address to the road network at the nearest intersection

near_intersection_query = """
CALL apoc.periodic.iterate(
  'MATCH (p:Address) WHERE NOT EXISTS ((p)-[:NEAREST_INTERSECTION]->(:Intersection)) RETURN p',
  'CALL {
  WITH p
  MATCH (i:Intersection)
  USING INDEX i:Intersection(location)
  WHERE point.distance(i.location, p.location) < 200

  WITH i
  ORDER BY point.distance(p.location, i.location) ASC 
  LIMIT 1
  RETURN i
}
WITH p, i

MERGE (p)-[r:NEAREST_INTERSECTION]->(i)
SET r.length = point.distance(p.location, i.location)
RETURN COUNT(p)',
  {batchSize:1000, parallel:false})
"""

# Create a full text index to enable search in our web app

full_text_query = "CREATE FULLTEXT INDEX search_index IF NOT EXISTS FOR (p:PointOfInterest|Address) ON EACH [p.name, p.full_address]"

In [ ]:
def enrich_addresses(tx):
    results = tx.run(add_addresses_query)        
    results = tx.run(near_intersection_query)


In [ ]:
# with driver.session() as session:
#     session.execute_write(enrich_addresses)

In [ ]:
# with driver.session() as session:
#     results = session.execute_write(lambda tx: tx.run(full_text_query))


### Query with address

```
MATCH (a:Address)-[:NEAREST_INTERSECTION]->(source:Intersection)
WHERE a.full_address CONTAINS "410 E 5TH AVE SAN MATEO, CA"
MATCH 
  (poi:Address)-[:NEAREST_INTERSECTION]->(dest:Intersection) 
WHERE poi.full_address CONTAINS "111 5TH AVE"
CALL apoc.algo.dijkstra(source, dest, "ROAD_SEGMENT", "length") 
YIELD weight, path
RETURN **
```


```
CALL db.index.fulltext.queryNodes("search_index", $searchString) 
YIELD node, score
RETURN coalesce(node.name, node.full_address) AS value, score, labels(node)[0] AS label, node.id AS id
ORDER BY score DESC LIMIT 25
```

```
MATCH (to {id: $dest})-[:NEAREST_INTERSECTION]->(source:Intersection) 
MATCH (from {id: $source})-[:NEAREST_INTERSECTION]->(target:Intersection)
CALL apoc.algo.dijkstra(source, target, 'ROAD_SEGMENT', 'length')
YIELD path, weight
RETURN [n in nodes(path) | [n.location.latitude, n.location.longitude]] AS route
```
